In [ ]:
import scipy
import matplotlib.pyplot as plt
import os
import ffmpeg
import moviepy.editor as mp
from scipy.io.wavfile import read
import numpy as np
from scipy.fft import fft, ifft
import math
import numpy as np
from pathlib import Path
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from typing import Tuple, Dict, Union
import ffmpeg

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
DATA_FOLDER = os.path.join(ROOT_DIR, "data")

In [ ]:
video_waltter_path = os.path.join(DATA_FOLDER, "example_waltter.MOV")
video_vikture_path = os.path.join(DATA_FOLDER, "example_vikture.MOV")

video_waltter_e15_path = os.path.join(DATA_FOLDER, "example_waltter_early_15s.MOV")
video_vikture_e15_path = os.path.join(DATA_FOLDER, "example_vikture_early_15s.MOV")

video_waltter_l15_path = os.path.join(DATA_FOLDER, "example_waltter_late_15s.MOV")
video_vikture_l15_path = os.path.join(DATA_FOLDER, "example_vikture_late_15s.MOV")

video_waltter_e30_path = os.path.join(DATA_FOLDER, "example_waltter_early_30s.MOV")
video_vikture_e30_path = os.path.join(DATA_FOLDER, "example_vikture_early_30s.MOV")

In [ ]:
# Get delay from synchronize_audio notebook
delay = 14.839274376417233

In [ ]:
def get_video_info(video_path) -> Dict[str, Union[int, float, str]]:
    probe = ffmpeg.probe(video_path)
    file_path = str(probe['format']['filename'])
    video_stream = next((stream for stream in probe['streams'] if stream['codec_type'] == 'video'), None)
    width = int(video_stream['width'])
    height = int(video_stream['height'])
    duration = float(probe['format']['duration'])
    # Frame rate removed in notebook because cannot import relative local utils
    
    return {
        "file_path" : file_path,
        "frame_height": height,
        "frame_width": width,
        #"frame_rate": frame_rate,
        "duration": duration
    }

In [ ]:
print(get_video_info(video_waltter_path))

In [ ]:
def extract_filename_without_extension(path) -> str:
    filename = Path(path).stem
    return filename


def split_file_from_extension(path, without_dot=True) -> Tuple[str, str]:
    filename, extension = os.path.splitext(path)
    if without_dot:
        extension = extension.replace('.', '').lower()
    return filename, extension

def extract_folder_from_path(path) -> str:
    return os.path.dirname(path)


# Separate path to three parts: folder, filename, file extension
def separate_path_to_parts(path) -> Tuple[str, str, str]:
    filename = extract_filename_without_extension(path)
    file_extension = split_file_from_extension(path)[1]
    folder = extract_folder_from_path(path)
    
    return folder, filename, file_extension


print(separate_path_to_parts(video_waltter_path))

In [ ]:
# If delay is positive, audio1 needs to be delayed and negative if audio2 needs to be delayed
def synchronize_videos(video1_path, video2_path, delay: float, output_path=None):
    video1_info = get_video_info(video1_path)
    video2_info = get_video_info(video2_path)
    video1_path_separated = separate_path_to_parts(video1_path)
    video2_path_separated = separate_path_to_parts(video2_path)

    final_duration = min(video1_info['duration'], video2_info['duration']) - abs(delay)
    
    if delay >= 0:
        video1_start_time = delay
        video1_end_time = final_duration + delay
        video1_name = os.path.join(video1_path_separated[0], f"{video1_path_separated[1]}_synchronized.{video1_path_separated[2]}")
        video2_start_time = 0
        video2_end_time = final_duration
        video2_name = os.path.join(video2_path_separated[0], f"{video2_path_separated[1]}_synchronized.{video2_path_separated[2]}")

        ffmpeg_extract_subclip(video1_path, video1_start_time, video1_end_time, targetname=video1_name)
        ffmpeg_extract_subclip(video2_path, video2_start_time, video2_end_time, targetname=video2_name)

    else:
        video1_start_time = 0
        video1_end_time = final_duration
        video1_name = os.path.join(video1_path_separated[0], f"{video1_path_separated[1]}_synchronized.{video1_path_separated[2]}")

        video2_start_time = delay
        video2_end_time = final_duration + delay
        video2_name = os.path.join(video2_path_separated[0], f"{video2_path_separated[1]}_synchronized.{video2_path_separated[2]}")

        ffmpeg_extract_subclip(video1_path, video1_start_time, video1_end_time, targetname=video1_name)
        ffmpeg_extract_subclip(video2_path, video2_start_time, video2_end_time, targetname=video2_name)


In [ ]:
synchronize_videos(video_waltter_path, video_vikture_l15_path, delay=delay)